<a href="https://colab.research.google.com/github/LinaMariaCastro/curso-ia-para-economia/blob/main/clases/3_Analisis_y_visualizacion_datos/2_Combinacion_Exportacion_Datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Inteligencia Artificial con Aplicaciones en Economía I**

- 👩‍🏫 **Profesora:** [Lina María Castro](https://www.linkedin.com/in/lina-maria-castro)  
- 📧 **Email:** [lmcastroco@gmail.com](mailto:lmcastroco@gmail.com)  
- 🎓 **Universidad:** Universidad Externado de Colombia - Facultad de Economía

# 🛢️**Métodos de importación, combinación y exportación de datos**

**Objetivos de Aprendizaje:**

Al finalizar este notebook, serás capaz de:
1.  **Conectar y extraer datos** desde bases de datos SQL (SQLite) y archivos semi-estructurados como JSON.
2.  **Combinar múltiples datasets** de forma eficiente utilizando las funciones `concat` y `merge` de pandas.
3.  **Exportar tus DataFrames resultantes** a diversos formatos para su uso posterior.

---

## Importar librerías

In [ ]:
import os
import numpy as np
import pandas as pd

## Mejorar visualización de los dataframes

In [ ]:
# Que muestre todas las columnas
pd.options.display.max_columns = None
# En los dataframes, mostrar los float con dos decimales
pd.options.display.float_format = '{:,.2f}'.format

## Establecer la ruta de los datasets

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive')

In [ ]:
path = '/content/drive/MyDrive/2025_ii_curso_ia_economia/datasets'

In [ ]:
# Para establecer el directorio de los archivos
os.chdir(path)

---

## Importando Datos de Diversas Fuentes

Los datos económicos no siempre vienen en un `csv` o `Excel`. A menudo, necesitamos interactuar con sistemas más complejos como bases de datos o formatos web como JSON.

### Importación desde una Base de Datos SQL

Las bases de datos son la columna vertebral de cualquier sistema de información. Bancos, entidades gubernamentales y empresas almacenan sus registros en bases de datos relacionales. Aprender a extraer datos de ellas es una habilidad crucial.

**¿Qué es una Base de Datos Relacional?**🗄️

Imagina que administras una tienda. Podrías registrar todas tus ventas en una **gigantesca hoja de Excel**. Por cada venta, anotarías el nombre del cliente, su email, el nombre del producto, su precio, la fecha, etc.

Pronto notarías los **problemas**:

- **Redundancia:** Si Ana Pérez compra 10 veces, anotas su nombre y email 10 veces.

- **Riesgo de Errores**: En una de esas 10 veces, podrías escribir "Ana Peres". Ahora, para el sistema, son dos clientas diferentes, y tus análisis serán incorrectos.

- **Ineficiencia:** El archivo se vuelve enorme y lento.

**Una base de datos relacional resuelve esto de forma elegante.** En lugar de una tabla gigante, creas tablas que se relacionan entre sí:

- **Tabla Clientes:** Solo guarda información de clientes. Cada cliente tiene un "id" único: cliente_id, el cuál podría ser su número de cédula.

- **Tabla Productos:** Solo guarda información de productos. Cada producto tiene un id único: producto_id.

- **Tabla Ventas:** Registra un único id para la venta (venta_id), el cual podría ser el número de factura, la fecha de venta, la cantidad de productos y lo más importante: cliente_id y producto_id. En lugar de repetir todos los datos de clientes y productos vendidos, esta tabla solo registra los ids de estos.

**La "relación" es el vínculo que se crea mediante estos IDs.** La tabla Ventas no sabe el nombre del cliente, pero sabe su cliente_id, y con este puede ir a buscar los detalles en la tabla Clientes.

**En resumen, una base de datos relacional es un sistema para almacenar datos en tablas separadas y conectadas por llaves, lo que elimina la redundancia, garantiza la consistencia y permite cruzar información de forma muy poderosa.**

![Ejemplo Base de Datos Realcional](https://drive.google.com/uc?id=1o0XV423UuWkbFzS6TUS_BfPrc12Bs2ln)

**¿Qué es SQL?** 🗣️

**SQL (Structured Query Language) es el idioma que se usa para pedirle al sistema que busque, filtre y combine información para generar reportes.**

No es un lenguaje de programación como Python, sino un lenguaje de consulta. **Su objetivo es solicitar, manipular y gestionar datos dentro de una base de datos relacional.**

La estructura de una petición básica en SQL es muy intuitiva y se parece a cómo pensarías la pregunta en español. Las "palabras clave" o **comandos más básicos** son:

- **SELECT:** Elige las columnas que quieres ver.

- **FROM:** Indica de qué tabla quieres sacar los datos.

- **WHERE:** Aplica un filtro a las filas (observaciones) para obtener solo las que cumplen una condición.

Pregunta de negocio simple: **"Quiero ver las ventas donde se ha vendido más de 5 unidades."**

In [ ]:
SELECT * # Tráeme todas las columnas
FROM Ventas # de la tabla Ventas
WHERE cantidad > 5; # pero solo las filas (registros) donde se vendieron más de 5 unidades

Aprender SQL es fundamental para un economista porque te da acceso directo a las fuentes primarias de datos en empresas y gobiernos, sin depender de que alguien más te exporte un archivo .csv.

Para el ejemplo, usaremos **SQLite**, un Sistema Gestor de Bases de Datos Relacionales sencillo que viene incluido con Python.

**Nota:** Un **Sistema Gestor de Bases de Datos Relacionales** (SGBDR), o RDBMS por sus siglas en inglés, es el **software especializado que te permite crear, consultar, actualizar y administrar una base de datos relacional.** Se encarga de la seguridad, la integridad, la eficiencia de las consultas y el acceso concurrente, permitiendo que los datos sean un activo fiable y utilizable para una organización.

Otros SGBDR son PostgreSQL y MySQL.

In [ ]:
import sqlite3

In [ ]:
# Paso 1: Establecer una conexión con la base de datos SQLite
conn = sqlite3.connect('economia_colombia.db')

In [ ]:
# Paso 2: Extraer los datos usando una consulta SQL
# Consulta: Selecciona todas las columnas de la tabla 'indicadores_macro' (trae todos los datos)

query = "SELECT * FROM indicadores_macro"
df_from_sql = pd.read_sql_query(query, conn)

print("Datos extraídos de la base de datos:")
display(df_from_sql)

In [ ]:
# Otra consulta: "Selecciona todas las columnas de la tabla 'indicadores_macro' donde el año sea mayor a 2021"

query2 = "SELECT * FROM indicadores_macro WHERE año > 2021"
df_from_sql2 = pd.read_sql_query(query2, conn)

print("Datos extraídos de la base de datos:")
display(df_from_sql2)

In [ ]:
# Paso 3: cerrar la conexión
conn.close()

---

### Importación de Archivos JSON

**JSON (JavaScript Object Notation)** es el lenguaje de la web. Muchas APIs y fuentes de datos abiertos (como las del gobierno) entregan la información en este formato, que es flexible pero puede ser anidado.

Vamos a utilizar un ejemplo que simula datos de población y PIB per cápita para Colombia.

In [ ]:
import json

Cargamos el archivo .json

In [ ]:
# Esta línea lee el archivo JSON y lo convierte en un objeto de Python
json_data=json.load(open('ejemplo_poblacion_pib_pc.json','r'))
json_data

En pandas, podemos crear un dataframe directamente a partir de un objeto tipo JSON, ya que este archivo es equivalente a un diccionario de Python. Sin embargo, puede ocurrir que no obtengamos la tabla que queremos.

In [ ]:
data = pd.DataFrame(json_data)
data

Para convertirlo a DataFrame, a menudo es necesario "normalizar" los datos anidados.

En este ejemplo, nos interesa la lista que está bajo la llave "datos_poblacionales", por lo que en lugar de convertir directamente a un dataframe, debemos utilizar json_normalize e indicar el record_path que queremos.

In [ ]:
df_from_json = pd.json_normalize(json_data, record_path='datos_poblacionales')
df_from_json

El código anterior solo extrae la tabla anidada de datos_poblacionales. ¿Pero qué pasa con la información del nivel superior, como "pais": "Colombia"?

Para eso, json_normalize tiene el parámetro meta, en el cual podemos indicar qué otras llaves del nivel superior queremos añadir como columnas en nuestro dataFrame final.

In [ ]:
df_completo = pd.json_normalize(json_data, record_path='datos_poblacionales', meta=['pais'])
df_completo

---

## Unir Datos con `pd.concat` y `pd.merge`

Rara vez un proyecto de análisis económico se basa en una sola fuente de datos, por tanto, es muy importante saber combinar datasets de manera lógica y correcta.

### Concatenación con `pd.concat()`

`concat` nos permite "apilar" DataFrames uno encima del otro (eje 0) o uno al lado del otro (eje 1). Su uso más común es para agregar nuevas filas al DataFrame.

**Ejemplo:** Eres un analista del DANE, de la Coordinación de Comercio Internacional, y tienes el archivo de las exportaciones de bienes de julio 2024. Al mes siguiente, recibes el archivo de agosto. `concat` es la acción de tomar las hojas del nuevo reporte y añadirlas al final del reporte del mes anterior. Esto se suele hacer con los archivos de enero a diciembre para construir un único archivo consolidado del año.

In [ ]:
df_expo_jul = pd.read_excel('expo_bienes_2024/07_Exportaciones_2024_Julio.xlsx')
df_expo_jul.head(3)

In [ ]:
df_expo_ago = pd.read_excel('expo_bienes_2024/08_Exportaciones_2024_Agosto.xlsx')
df_expo_ago.head(3)

In [ ]:
df_expo_jul.shape

In [ ]:
df_expo_ago.shape

In [ ]:
# Para concatenar un dataframe debajo del otro (axis=0)
df_expo_2024 = pd.concat([df_expo_jul, df_expo_ago], axis = 0, ignore_index=True)
df_expo_2024.head(3)

In [ ]:
df_expo_2024.tail(3)

In [ ]:
df_expo_2024.shape

In [ ]:
# Para concatenar un dataframe al lado del otro, se coloca axis = 1
df_expo_2024_hor = pd.concat([df_expo_jul, df_expo_ago], axis = 1)
df_expo_2024_hor.head(3)

In [ ]:
df_expo_2024_hor.shape

In [ ]:
df_expo_2024_hor.tail(3)

### Cruce de Información con `pd.merge()`

`merge` es una de las operaciones más poderosas. Emula los `JOIN` de SQL y nos permite combinar DataFrames basándose en el valor de una o más columnas en común (llaves o "keys").

**Ejemplo:** El Banco de la República te contrata. Te entregan una tabla con el Indicador de Seguimiento a la Economía (ISE) para cada mes. Por otro lado, la Bolsa de Valores de Colombia te da una tabla con el volumen de negociación del índice COLCAP, también para cada mes. `merge` es la operación de tomar estas dos tablas y, usando la columna "mes" como el punto de conexión, crear una nueva tabla más rica que contenga tanto el ISE como el volumen del COLCAP en la misma fila para cada mes.

Vamos a explorar los 4 tipos de `merge` más comunes: `inner`, `left`, `right`, y `outer`.

![Tipos de merge](https://drive.google.com/uc?id=1JQ_QH90nibxiBXoZezP4GgtMFNMp7gzV)

In [ ]:
# Para el ejemplo, vamos a crear dos dataframes

# DataFrame 1: Datos de población por país
df_poblacion = pd.DataFrame({
    'codigo_pais': ['COL', 'MEX', 'PER', 'ECU'],
    'poblacion_millones': [52.1, 128.9, 34.0, 18.0]
})

# DataFrame 2: Datos de Llegada de Inversión Extranjera Directa (IED)
# Nota: No tenemos datos para Ecuador (ECU), pero sí para Argentina (ARG), que no está en el otro DF.
df_ied = pd.DataFrame({
    'iso_code': ['COL', 'MEX', 'PER', 'ARG'],
    'ied_millones_usd': [17048, 35292, 11738, 15055]
})

print("Población:")
display(df_poblacion)
print("IED:")
display(df_ied)

#### MERGE TIPO 1: INNER JOIN (Intersección)

Solo nos devuelve las filas donde la llave ('codigo_pais'/'iso_code') existe en AMBOS DataFrames.

In [ ]:
df_inner = pd.merge(df_poblacion, df_ied, left_on='codigo_pais', right_on='iso_code', how='inner')

print("\n--- INNER JOIN ---")
print("Resultado: Solo países presentes en ambos datasets.")
display(df_inner)


#### MERGE TIPO 2: LEFT JOIN

Mantiene TODAS las filas del DataFrame de la izquierda ('df_poblacion') y trae los datos del de la derecha si la llave coincide. Si no, pone NaN.

In [ ]:
df_left = pd.merge(df_poblacion, df_ied, left_on='codigo_pais', right_on='iso_code', how='left')

print("\n--- LEFT JOIN ---")
print("Resultado: Todos los países de 'población', con datos de IED si están disponibles.")
display(df_left)

#### MERGE TIPO 3: RIGHT JOIN
Mantiene TODAS las filas del DataFrame de la derecha ('df_ied') y trae los datos del de la izquierda si la llave coincide.

In [ ]:
df_right = pd.merge(df_poblacion, df_ied, left_on='codigo_pais', right_on='iso_code', how='right')

print("\n--- RIGHT JOIN ---")
print("Resultado: Todos los países de 'IED', con datos de población si están disponibles.")
display(df_right)

#### MERGE TIPO 4: OUTER JOIN (Unión Completa)

Mantiene TODAS las filas de AMBOS DataFrames. Rellena con NaN donde no hay correspondencia.

In [ ]:
df_outer = pd.merge(df_poblacion, df_ied, left_on='codigo_pais', right_on='iso_code', how='outer')

print("\n--- OUTER JOIN ---")
print("Resultado: Todos los países de ambos datasets.")
display(df_outer)

#### Ejercicios

1. Tienes el siguiente dataframe `df_econ` con datos económicos para Colombia, México, Perú y Brasil y el dataframe `df_poblacion` que creamos previamente. Usando un `merge`, crea un nuevo DataFrame llamado `df_final` que contenga el PIB per cápita, la inflación, las exportaciones y la población para cada país. ¿Qué tipo de `join` es el más apropiado si quieres mantener todos los datos de `df_econ`?

In [ ]:
# Datos económicos por país
df_econ = pd.DataFrame({
    'pais_iso': ['COL', 'MEX', 'PER', 'BRA'],
    'PIB_per_capita': [7917, 13954, 8316, 10296],
    'Inflacion': [6.61, 4.72, 2.01, 4.83],
    'Exportaciones': [49600, 680790, 74700, 337000]
})
df_econ

2. Tenemos estas dos tablas, una con países y su PIB, y otra con países y su índice de desarrollo humano (IDH). No todos los países están en ambas tablas. Discutan con su compañero qué tabla resultante obtendrían con un inner join vs un outer join.

In [ ]:
# DataFrame 1: Datos de PIB por país
df_pib = pd.DataFrame({
    'pais': ['BRA', 'ARG', 'PER', 'CHL'],
    'pib_millones_usd': [2188, 604, 283, 329]
})

# DataFrame 2: Datos IDH por país
df_idh = pd.DataFrame({
    'pais': ['BRA', 'CHL', 'COL', 'PER', 'MEX'],
    'IDH': [0.786, 0.878, 0.788, 0.794, 0.789],
    'Categoría': ['Alto', 'Muy alto', 'Alto', 'Alto', 'Alto']
})

print("PIB:")
display(df_pib)
print("IDH:")
display(df_idh)

3.  He intentado hacer un merge entre estos dos DataFrames, pero Python me da un error. ¿Quién puede ver por qué está fallando el código?

In [ ]:
# DataFrame 1: Datos de PIB per cápita por país
df_pib_pc = pd.DataFrame({
    'país_iso': ['COL', 'MEX', 'PER', 'BRA'],
    'PIB_per_capita': [7917, 13954, 8316, 10296]
})

# DataFrame 2: Datos de inflación por país
df_inflacion = pd.DataFrame({
    'pais_iso': ['COL', 'MEX', 'PER', 'BRA'],
    'Inflacion': [6.61, 4.72, 2.01, 4.83]
})

print("PIB per cápita:")
display(df_pib_pc)
print("Inflación:")
display(df_inflacion)

In [ ]:
df_merge = pd.merge(df_pib_pc, df_inflacion, on='pais_iso', how='inner')
df_merge

In [ ]:
# Solución

4. Y en este merge, ¿qué está fallando?

In [ ]:
# DataFrame 1: Datos de PIB en USD por municipio
df_pib_usd = pd.DataFrame({
    'cod_municipio': ['85424', '16784', '13968', '15978'],
    'PIB_usd': [417, 1848, 283, 2188]
})

# DataFrame 2: Datos de poblacion por municipio
df_poblacion = pd.DataFrame({
    'cod_municipio': [85424, 16784, 13968, 15978],
    'Población': ['49600', '680790', '74700', '337000']
})

print("PIB en USD:")
display(df_pib_usd)
print("Población:")
display(df_poblacion)

In [ ]:
df_combinado = pd.merge(df_pib_usd, df_poblacion, on='cod_municipio', how='inner')
df_combinado

In [ ]:
# Solución

#### Pregunta de discusión

¿Qué harían si al intentar unir dos tablas del gobierno por "nombre del municipio", descubren que "Bogotá D.C." en una tabla se llama "Bogota" y en la otra se llama Bogotá? ¿Cómo afecta esto a los diferentes tipos de join?

#### Ejemplos aplicados a Economía

- **Scoring de Crédito:** Unir el historial de crédito de un cliente (desde una base de datos SQL) con sus datos de redes sociales (desde una API que devuelve JSON) para crear un perfil de riesgo más completo.

- **Análisis de Política Pública:** Combinar datos de ejecución presupuestal de municipios (archivo CSV del gobierno) con indicadores de pobreza del DANE (tabla en Excel) con el fin de evaluar el impacto del gasto.

- **Investigación Macroeconómica:** Fusionar series de tiempo 2000-2025 sobre la inflación y el desempleo de todos los países de Latam (varios archivos de Excel) con el fin de crear un panel de datos y analizar el comportamiento de estas dos variables en el tiempo.

---

## Casos especiales de importación de archivos

### Llamar todos los archivos dentro de una carpeta y concatenarlos

**Ejemplo:** Eres analista del DANE en la Coordinación de Comercio Internacional y cuentas con una carpeta que contiene los archivos de exportaciones de bienes correspondientes a julio, agosto y septiembre de 2024. Necesitas consolidarlos en un único archivo del tercer trimestre de 2024, pero no deseas cargar cada archivo de forma manual, sino que el programa procese automáticamente todos los archivos Excel ubicados en la carpeta que especifiques.

In [ ]:
import glob

**Nota:** La librería glob se usa para buscar y listar archivos en carpetas que cumplen un patrón en su nombre (comodines como *, ?). Es muy útil cuando tienes muchos archivos CSV, Excel, TXT, etc. en una carpeta y quieres procesarlos en un solo paso.

In [ ]:
# Buscar todos los archivos Excel en la carpeta
archivos = glob.glob('expo_bienes_2024/*.xlsx')

# Lista para guardar DataFrames temporales
lista_dfs = []

for archivo in archivos:
    try:
        temp = pd.read_excel(archivo)
        lista_dfs.append(temp)

        # Verificar columnas para identificar si algún archivo tiene más de las que debería
        if len(temp.columns) > 72:
            print(archivo, len(temp.columns))

    except Exception as e:
        print(f"Error leyendo {archivo}: {e}")

# Concatenar todos los DataFrames en uno solo
df_expo = pd.concat(lista_dfs, ignore_index=True)
df_expo.head(3)

In [ ]:
df_expo.shape

In [ ]:
# Para comprobar los meses que quedaron en el dataframe
df_expo['FECHA_PROCESO'].value_counts(dropna=False)

### Cargar una base muy pesada (varias gigas) por chunks

**Ejemplo**: Trabajas en la Dirección de Estudios Económicos de la Superintendencia de Sociedades y necesitas cargar y analizar la base de datos de las 10.000 empresas más grandes del país. Sin embargo, la carga completa está tardando varias horas y tu jefe te ha solicitado el análisis para las 4 de la tarde. Para optimizar el proceso y reducir significativamente el tiempo, decides realizar la carga por chunks.

In [ ]:
# Importamos la librería time que nos ayudará a medir el tiempo de ejecución
from time import time

In [ ]:
tiempo_inicio = time()
df_ssc = pd.read_csv('Supersociedades_2023.csv', encoding ='utf-8', dtype=str)
df_ssc_reducido = df_ssc[['NIT', 'RAZON_SOCIAL', 'DEPARTAMENTO', 'MUNICIPIO',
                            'CIIU', 'DESCRIPCION_CIIU']]
tiempo_termino = time()
tiempo_carga_completa= tiempo_termino - tiempo_inicio
print(f"Tiempo carga completa: {tiempo_carga_completa:.6f} segs")

In [ ]:
df_ssc_reducido.head(3)

In [ ]:
df_ssc_reducido.shape

In [ ]:
tiempo_inicio = time()
df_chunk = pd.read_csv('Supersociedades_2023.csv', encoding ='utf-8', chunksize=1000)
chunk_list = []
for chunk in df_chunk:
    chunk_reducido = chunk[['NIT', 'RAZON_SOCIAL', 'DEPARTAMENTO', 'MUNICIPIO',
                            'CIIU', 'DESCRIPCION_CIIU']]

    # Añadir los datos a la lista de objetos
    chunk_list.append(chunk_reducido)

# Concatenar los datos filtrados en un DataFrame
df_ssc_reducido2 = pd.concat(chunk_list)
tiempo_termino = time()
tiempo_chunks = tiempo_termino - tiempo_inicio
print(f"Tiempo carga por chunks: {tiempo_chunks:.6f} segs")

In [ ]:
df_ssc_reducido2.head()

In [ ]:
df_ssc_reducido2.shape

---

## Guardar / exportar datos

Una vez que hemos limpiado, combinado y analizado nuestros datos, el último paso es guardarlos. Esto nos permite compartir resultados o usarlos en otros programas (como R, Stata, o Tableau).

### Guardar en CSV (Comma-Separated Values) - el más común

In [ ]:
df_ssc_reducido.to_csv('Descargas/Supersociedades_2023_reducido.csv', encoding ='utf-8', index = False)

### Guardar en TXT

In [ ]:
df_ssc_reducido.to_csv('Descargas/Supersociedades_2023_reducido.txt', sep="|", decimal=',', encoding ='utf-8', index= False)

### Guardar en JSON

El formato 'records' es muy útil para la web y APIs. Significa que cada fila del DataFrame se convertirá en un diccionario (JSON object) dentro de una lista.

indent=4: Agrega sangría de 4 espacios en el JSON para que sea más legible.
Si no se coloca, todo el JSON queda en una sola línea.

In [ ]:
df_final.to_json('Descargas/datos_macro_final.json', orient='records', indent=4)

### Guardar en Excel

In [ ]:
df_ssc_reducido.to_excel("Descargas/Supersociedades_2023_reducido.xlsx", index=False)

#### Guardar más de un dataframe en Excel

In [ ]:
with pd.ExcelWriter('Descargas/Ejemplo.xlsx') as writer:
    df_ssc_reducido.to_excel(writer, sheet_name='SSC', index=False)
    df_final.to_excel(writer, sheet_name='Macro', index=False)

#### Guardar un dataframe en un Excel que ya existe como una hoja adicional (mode='a)

Si la hoja ya existe, la reemplaza.

In [ ]:
with pd.ExcelWriter('Descargas/Ejemplo.xlsx', mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    df_expo_2024.to_excel(writer, sheet_name='Expo', index = False)